In [9]:
# Import the Python SDK
import google.generativeai as genai
from dotenv import load_dotenv
import os
import pandas as pd

In [3]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI-API-KEY")

In [5]:
genai.configure(api_key=GEMINI_API_KEY)

In [6]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [8]:
response = model.generate_content("Can you produce useful insights from bunch of reviews about a product which might help me to analyse the product.")
print(response.text)

Please provide me with the reviews! I need the actual text of the reviews to analyze them. 

Once you provide me with the reviews, I can help you with insights by:

**1. Identifying common themes and patterns:**  I can analyze the reviews to find recurring positive and negative comments, identifying key strengths and weaknesses of the product.

**2. Extracting specific features mentioned:**  I can pinpoint which features are most frequently praised or criticized, helping you understand what users value most.

**3. Determining customer sentiment:** I can assess the overall sentiment expressed in the reviews (positive, negative, neutral), giving you a sense of customer satisfaction.

**4. Categorizing reviews by user type:**  I can identify different types of users (e.g., first-time buyers, experienced users) and their specific feedback, helping you understand different perspectives.

**5. Generating a summary of key findings:** I can create a concise summary of the most important insigh

In [10]:
reviews = pd.read_csv("./Review_Data.csv")

In [18]:
reviews.head(2)

,Unnamed: 0,title,rating,content
0,0,"\r\nPremium phone at budget price, loved it\r\n",5,"Points to know before buying, got it for 17k: ..."
1,1,\r\nBest 16k phone\r\n,5,Absolute value for money mobile phone. Snappy ...


In [30]:
reviews = reviews.drop(reviews.columns[0], axis = 1)
reviews.head()

,title,rating,content
0,"\r\nPremium phone at budget price, loved it\r\n",5,"Points to know before buying, got it for 17k: ..."
1,\r\nBest 16k phone\r\n,5,Absolute value for money mobile phone. Snappy ...
2,"\r\nOverall very good phone, 25w is -ve comapr...",4,Everything about this phone is great: the disp...
3,\r\nGreat product with great deal\r\n,5,Bought it with exchange offer during prime day...
4,\r\nDisplay and sound is superb. Camera is jus...,3,Display is best as usual for samsung phones. S...


In [36]:
reviews.rename(columns = {'content': 'body'}, inplace = True)
reviews.head()

,title,rating,body
0,"\r\nPremium phone at budget price, loved it\r\n",5,"Points to know before buying, got it for 17k: ..."
1,\r\nBest 16k phone\r\n,5,Absolute value for money mobile phone. Snappy ...
2,"\r\nOverall very good phone, 25w is -ve comapr...",4,Everything about this phone is great: the disp...
3,\r\nGreat product with great deal\r\n,5,Bought it with exchange offer during prime day...
4,\r\nDisplay and sound is superb. Camera is jus...,3,Display is best as usual for samsung phones. S...


In [60]:
reviews

,title,rating,body
0,"\r\nPremium phone at budget price, loved it\r\n",5,"Points to know before buying, got it for 17k: ..."
1,\r\nBest 16k phone\r\n,5,Absolute value for money mobile phone. Snappy ...
2,"\r\nOverall very good phone, 25w is -ve comapr...",4,Everything about this phone is great: the disp...
3,\r\nGreat product with great deal\r\n,5,Bought it with exchange offer during prime day...
4,\r\nDisplay and sound is superb. Camera is jus...,3,Display is best as usual for samsung phones. S...
...,...,...,...
75,Awesome!,5,Great buy if bought on offers...
76,Best phone,5,Worth buy
77,Good phone,5,Very good phone with best quality camera\r\nBe...
78,Heating problem,3,Facing heating problem on normal phone usage. ...


In [62]:
rvwStruct = {"reviews": []}

for i in range(len(reviews['title'])):
    inst = reviews.iloc[i]
    rvw = {"rating": inst['rating'], "title": inst['title'], "body": inst['body']}
    rvwStruct['reviews'].append(rvw)

In [64]:
rvwStruct['reviews'][:10]

[{'rating': 5,
  'title': '\r\nPremium phone at budget price, loved it\r\n',
  'body': "Points to know before buying, got it for 17k: 1. Ui and updates 4.5/5, it's smooth and responsive no complains 2. Camera 4.6/5, comparable with Samsung midrange phones 3. Battery 4.2/5, I charge my phone upto 80% It takes around 45 min for..."},
 {'rating': 5,
  'title': '\r\nBest 16k phone\r\n',
  'body': 'Absolute value for money mobile phone. Snappy interface, monstrous battery(Screenshot attached - Without SIM). Good set of cameras. Clean One UI. Samsung wallet feature. Good in hand feel.'},
 {'rating': 4,
  'title': '\r\nOverall very good phone, 25w is -ve comapred to competitors.\r\n',
  'body': 'Everything about this phone is great: the display, camera, speed, and battery life. The only drawback is the slow charging speed it charges at 25W, which is quite low compared to the competition, as even phones priced at 10,000 INR now feature 33W fast charging. If this...'},
 {'rating': 5,
  'title':

In [71]:
prompt = """You are an advanced language model tasked with analyzing customer reviews for a specific product. The reviews include a rating out of 5, a title, and the main body of the review. Your objective is to extract meaningful insights that will help a potential buyer make an informed decision about whether to purchase the product. Your analysis should provide:

Overall Sentiment Analysis:

Summarize the general sentiment of the reviews (positive, negative, or mixed).
Highlight any patterns or trends in the ratings (e.g., most reviews are 4 stars, consistent 5-star ratings, frequent 1-star complaints).
Pros:

Identify and list the most common positive aspects mentioned by reviewers.
Provide specific examples of features, qualities, or experiences that customers appreciated.
Cons:

Identify and list the most common negative aspects mentioned by reviewers.
Highlight any recurring issues, complaints, or dissatisfaction points raised by multiple reviewers.
Key Insights:

Extract any additional insights that do not fall strictly under pros or cons but are important for understanding the product (e.g., variations in product performance, unique use cases, or specific customer needs addressed).
Note any discrepancies between high ratings and negative comments or low ratings with positive comments.
Decision-Making Summary:

Provide a final summary that encapsulates the overall crux of the reviews.
Offer a balanced view that weighs the pros and cons to assist the potential buyer in making an informed decision.
Input Format:

Rating: [Rating out of 5]
Title: [Title of the review]
Body: [Body of the review]
Output Format:

Overall Sentiment Analysis:
[Summary of general sentiment and rating trends]
Pros:
[List of pros with examples]
Cons:
[List of cons with examples]
Key Insights:
[Additional important insights]
Decision-Making Summary:
[Balanced summary to aid in decision-making]
Here are the reviews:  
"""

In [72]:
type(prompt)

str

In [74]:
str(rvwStruct)

'{\'reviews\': [{\'rating\': 5, \'title\': \'\\r\\nPremium phone at budget price, loved it\\r\\n\', \'body\': "Points to know before buying, got it for 17k: 1. Ui and updates 4.5/5, it\'s smooth and responsive no complains 2. Camera 4.6/5, comparable with Samsung midrange phones 3. Battery 4.2/5, I charge my phone upto 80% It takes around 45 min for..."}, {\'rating\': 5, \'title\': \'\\r\\nBest 16k phone\\r\\n\', \'body\': \'Absolute value for money mobile phone. Snappy interface, monstrous battery(Screenshot attached - Without SIM). Good set of cameras. Clean One UI. Samsung wallet feature. Good in hand feel.\'}, {\'rating\': 4, \'title\': \'\\r\\nOverall very good phone, 25w is -ve comapred to competitors.\\r\\n\', \'body\': \'Everything about this phone is great: the display, camera, speed, and battery life. The only drawback is the slow charging speed it charges at 25W, which is quite low compared to the competition, as even phones priced at 10,000 INR now feature 33W fast charging

In [76]:
final_prompt = prompt + str(rvwStruct)
response = model.generate_content(final_prompt)

In [77]:
print(response.text)

## Samsung Galaxy M35 5G Review Analysis

**Overall Sentiment Analysis:**

The overall sentiment towards the Samsung Galaxy M35 5G is **positive**, with a majority of reviewers giving it a 4 or 5-star rating. The most common rating is 5 stars, indicating strong satisfaction among users. However, there are a few recurring complaints that pull the average down slightly. 

**Pros:**

* **Excellent Display:** Reviewers consistently praise the Super AMOLED display, highlighting its vibrant colors, sharp resolution, and smooth 120Hz refresh rate. 
    * *Example:* "The DISPLAY is really good, after all it's a SAMSUNG OLED panel, the colour and viewing angles are astonishing."
* **Long Battery Life:** The 6000mAh battery consistently receives positive feedback for its impressive longevity. 
    * *Example:* "Battery backup is just awesome. Call quality is also good wherein if you put call over..."
* **Good Performance:** Reviewers generally report smooth performance for everyday tasks and app